<a href="https://colab.research.google.com/github/hauduong05/NER_Extract_CV/blob/main/ExtractCV_CRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install sklearn_crfsuite

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [4]:
# read_data

header = ['Word', 'Pos', 'Chunking-Tag', 'Tag']

train_dt = pd.read_csv('/content/gdrive/MyDrive/NER/train-1.csv', sep= r'\t', names=header)
dev_dt = pd.read_csv('/content/gdrive/MyDrive/NER/dev-1.csv', sep = r'\t', names=header)
test_dt = pd.read_csv('/content/gdrive/MyDrive/NER/test-1.csv', sep = r'\t', names=header)

dt = pd.concat([train_dt, dev_dt, test_dt])
print(dt)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


                Word Pos Chunking-Tag Tag
0                Tạo   V         B-VP   O
1                các   L         B-NP   O
2       Module_Agent  Np         B-NP   O
3                  ,  CH            O   O
4         Callcenter  Np         B-NP   O
...              ...  ..          ...  ..
258568             ,  CH            O   O
258569     phần_cứng   N         B-NP   O
258570            và   C            O   O
258571      phần_mềm   N         B-NP   O
258572             .  CH            O   O

[1284092 rows x 4 columns]


In [5]:
# handing missing value

dt.isnull().sum()
dt = dt.fillna(method = 'ffill')

In [6]:
# group data into sentences

id = 1
sent_id = []

for i in dt['Word']:
  sent_id.append(id)
  if i == '.':
    id += 1
  
dt['Sentence'] = sent_id
func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                    s['Pos'].values.tolist(),
                                                    s['Tag'].values.tolist())]
sentences = dt.groupby('Sentence').apply(func)
sentences = sentences.values

In [7]:
# get features 

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
  return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
  return [label for word, pos, label in sent]
   

In [8]:
# prepare data

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

X_train, X_test , y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [9]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [10]:
# predict

y_pred = crf.predict(X_test)

f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f' f1 score = {f1_score:.3f} \n')

print(flat_classification_report(y_test, y_pred))

 f1 score = 0.952 

              precision    recall  f1-score   support

       B-COM       0.86      0.75      0.80      2502
       B-POS       0.84      0.72      0.78      2035
     B-SKILL       0.82      0.67      0.74      9237
       I-COM       0.87      0.84      0.86      7061
       I-POS       0.74      0.66      0.69      2695
     I-SKILL       0.71      0.50      0.59      3800
           O       0.97      0.98      0.98    227717

    accuracy                           0.95    255047
   macro avg       0.83      0.73      0.78    255047
weighted avg       0.95      0.95      0.95    255047

